In [3]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [4]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [5]:
_n_samples = 1  # number of tuples to create for iloc start_train:end_train:end_eval
_days_lookbacks = [60, 120, 30]
_days_eval = 10

In [6]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_train_eval_days = _3_random_slices(len_df_train, n_samples=_n_samples, days_lookback=max(_days_lookbacks), days_eval=_days_eval)
max_train_eval_days

[(74, 194, 204)]

In [7]:
def lookback_slices(max_slices, days_lookbacks, verbose=False):
  """Create 
  
  
  Returns perf_ranks_dict(dic. of dic. of symbols ranked in descending
    performance) and ranked_perf_ranks_dict(dic. of symbols ranked in
    descending frequency in a combined pool of symbols in perf_ranks_dict).

  Args:
      df_close(dataframe): dataframe of symbols' close with
        DatetimeIndex e.g. (['2016-12-19', ... '2016-12-22']), symbols as
        column names, and symbols' close as column values.
      days_lookbacks(list of positive integers): list of number of days to
      look-back, e.g. [15, 30], for performance calculation.
      n_top_syms(int): number of top symbols to keep in perf_ranks_dict 

  Return:
      perf_ranks_dict({dic): dic. of dic. of symbols ranked in descending
        performance.
        First dic keys are:
        'period' + str(days_lookbacks[0]), ... ,
        'period' + str(days_lookbacks[-1])
        Second dic keys are:
        'r_CAGR/UI', 'r_CAGR/retnStd' and 'r_retnStd/UI'
        e.g.:
        {
          period-15': {
                        'r_CAGR/UI':  ['HZNP', ... , 'CB'],
                        'r_CAGR/retnStd': ['BBW', ... , 'CPRX'],
                        'r_retnStd/UI':   ['ENR', ... , 'HSY']
                      },
          ... ,
          'period-60': {
                        'r_CAGR/UI':  ['WNC', ... , 'FSLR'],
                        'r_CAGR/retnStd': ['VCYT', ... , 'BERY'],
                        'r_retnStd/UI':   ['MYOV', ... , 'NSC']
                        }
        }
      ranked_perf_ranks_dict(dic): dic. of symbols ranked in descending
        frequency in a combined pool of symbols in perf_ranks_dict.  Key is
        'ranked_perf_ranks_period' + str(days_lookbacks), e.g.:
        {'ranked_perf_ranks_period[-15, -30]': ['HZNP', ... , 'NSC']}
  """  
  lb_slices = []
  days_lookbacks.sort()  # sort list of integers in ascending order
  for my_slice in max_slices:
    # lb_slices.append(my_slice)
    # if verbose:
    #   print(f'days: {days_lookbacks[-1]}, {my_slice}')
    for days in days_lookbacks:
      new_slice = (my_slice[1]-days, my_slice[1], my_slice[2])
      lb_slices.append(new_slice)
      if verbose:
        print(f'days: {days}, {new_slice}')      
    if verbose:    
     print('')
  return lb_slices

In [8]:
lb_slices = lookback_slices(max_slices=max_train_eval_days, days_lookbacks=_days_lookbacks, verbose=False)
lb_slices 

[(164, 194, 204), (134, 194, 204), (74, 194, 204)]

In [9]:
perf_ranks_dic = {}
ranked_perf_ranks_dic = {}

for lb_slice in lb_slices:
  start_train = lb_slice[0]
  end_train = lb_slice[1]
  start_eval = end_train
  end_eval = lb_slice[2]
  days_lookback = end_train - start_train
  days_eval = end_eval - start_eval
  print(f'lb_slice:      {lb_slice}')
  print(f'days_lookback: {days_lookback}')
  print(f'days_eval:     {days_eval}')    
  print(f'start_train:   {start_train}')
  print(f'end_train:     {end_train}')
  print(f'start_eval:    {start_eval}')
  print(f'end_eval:      {end_eval}')
  print('')
  _df = df_train.iloc[start_train:end_train]
  # perf_ranks_dic, ranked_perf_ranks_dic, l_tuples = _4_perf_ranks_1(_df, n_top_syms=5)    
  # perf_ranks[days_lookback] = perf_ranks
  # ranked_perf_ranks[days_lookback] = ranked_perf_ranks
  print('====', '\n')     

lb_slice:      (164, 194, 204)
days_lookback: 30
days_eval:     10
start_train:   164
end_train:     194
start_eval:    194
end_eval:      204

==== 

lb_slice:      (134, 194, 204)
days_lookback: 60
days_eval:     10
start_train:   134
end_train:     194
start_eval:    194
end_eval:      204

==== 

lb_slice:      (74, 194, 204)
days_lookback: 120
days_eval:     10
start_train:   74
end_train:     194
start_eval:    194
end_eval:      204

==== 



In [10]:
df_close = _df.copy()
n_top_syms = 5

In [11]:
perf_ranks, most_common_syms = _4_perf_ranks(df_close, n_top_syms=5)

In [14]:
# perf_ranks
most_common_syms

[('XOMA', 3),
 ('CDNA', 2),
 ('AVEO', 2),
 ('DVAX', 2),
 ('BLFS', 2),
 ('CYRX', 1),
 ('GDOT', 1),
 ('CTRA', 1),
 ('FTSM', 1)]

In [15]:
perf_ranks

{'period-120': {'r_CAGR/UI': array(['CDNA', 'AVEO', 'DVAX', 'XOMA', 'BLFS'], dtype=object),
  'r_CAGR/retnStd': array(['CDNA', 'AVEO', 'CYRX', 'BLFS', 'XOMA'], dtype=object),
  'r_retnStd/UI': array(['GDOT', 'DVAX', 'XOMA', 'CTRA', 'FTSM'], dtype=object)}}

In [ ]:
from trash import _4_perf_ranks_2
perf_ranks, most_common_syms = _4_perf_ranks_2(df_close, n_top_syms=5)

In [ ]:
for days_lookback in slices_dic:
  slices = slices_dic[days_lookback]
  print(days_lookback, slices)
  for my_slice in slices:
    start_train = my_slice[0]
    end_train = my_slice[1]
    start_val = end_train
    end_val = my_slice[2]
    print(f'start_train: {start_train}')
    print(f'end_train:   {end_train}')
    # print(f'start_val:   {start_val}')
    # print(f'end_val:     {end_val}')
    _df = df_train.iloc[start_train:end_train]
    perf_ranks, ranked_perf_ranks = _4_perf_ranks_1(_df, days_lookback=days_lookback)
    perf_ranks_dic[days_lookback] = perf_ranks
    ranked_perf_ranks_dic[days_lookback] = ranked_perf_ranks 
  print('')    

In [ ]:
perf_ranks_dic

In [ ]:
ranked_perf_ranks_dic

In [ ]:
# for my_slice in my_slices[0:2]:
for my_slice in my_slices:
  start_train = my_slice[0]
  end_train = my_slice[1]
  start_val = end_train
  end_val = my_slice[2]
  print(f'start_train: {start_train}')
  print(f'end_train: {end_train}')
  print(f'start_val: {start_val}')
  print(f'end_val: {end_val}')

  _df = df_train.iloc[start_train:end_train]
  # print(_df) 
  perf_ranks_dict, ranked_perf_ranks_dict = _4_perf_ranks(_df, days_lookbacks=[60])
  # print(perf_ranks_dict)
  # print(ranked_perf_ranks_dict)  

In [ ]:
# print(perf_ranks_dict)
trash = perf_ranks_dict
trash

In [ ]:
ranked_perf_ranks_dict